<h1> 1. Pre-Processing Dataset </h1> 

Pada Tugas kali ini, saya mengimplementasikan ANN (Jaringan Syaraf Tiruan) menggunakan sebuah library Keras untuk mengklasifikasikan tumor ke dalam tipe Ganas atau Jinak, jika diberikan ukuran tumornya. Pada hasil keluaran, akan memiliki kemungkinan tumor termasuk dalam kelas Maligna atau Benigna. Keseluruhan tugas dibagi menjadi 3 bagian yakni :
1. Pra-pemrosesan data dan analisis cepat
2. Membangun ANN
3. Membuat Prediksi

<h1> Loading Dataset </h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('data.csv')

### Library Numpy 
berfungsi untuk mengolah data dalam bentuk angka (array)
### Library Pandas
Berfungsi untuk membaca file dataset dalam bentuk .CSV
### Library Matplotlib 
berfungsi untuk membantu visualisasi dataset sehingga memudahkan untuk dipahami

In [2]:
def dataSetAnalysis(df):
    #view starting values of data set
    print("Dataset Head")
    print(df.head(3))
    print("=" * 30)
    
    # View features in data set
    print("Dataset Features")
    print(df.columns.values)
    print("=" * 30)
    
    # View How many samples and how many missing values for each feature
    print("Dataset Features Details")
    print(df.info())
    print("=" * 30)
    
    # view distribution of numerical features across the data set
    print("Dataset Numerical Features")
    print(df.describe())
    print("=" * 30)
    
    # view distribution of categorical features across the data set
    print("Dataset Categorical Features")
    print(df.describe(include=['O']))
    print("=" * 30)

dataSetAnalysis(dataset)

Dataset Head
         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38           122.8     1001.0   
1    842517         M        20.57         17.77           132.9     1326.0   
2  84300903         M        19.69         21.25           130.0     1203.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   

   ...  texture_worst  perimeter_worst  area_worst  smoothness_worst  \
0  ...          17.33            184.6      2019.0            0.1622   
1  ...          23.41            158.8      1956.0            0.1238   
2  ...          25.53            152.5      1709.0            0.1444   

   compactness_worst  concavity_worst  concave points_worst  symmetry_wo

Data memiliki total 33 kolom, kolom terakhir 'Tanpa Nama: 32' berisi semua nilai nol jadi data akan mengecualikannya. Selain itu label pada data adalah 'diagnosis'. data juga tidak akan menyertakan 'id' dalam set pelatihan, karena tidak berpengaruh pada klasifikasi. Jadi, data memiliki 30 fitur yang semuanya bertipe float64 dan tidak mengandung nilai yang hilang.

In [3]:
X = dataset.iloc[:,2:32] # [all rows, col from index 2 to the last one excluding 'Unnamed: 32']
y = dataset.iloc[:,1] # [all rows, col one only which contains the classes of cancer]

Perhatikan bahwa 'diagnosis' mengandung 'M' atau 'B' untuk mewakili tumor ganas atau jinak. Sistem mengenkodekannya menjadi 0 dan 1.

In [4]:
from sklearn.preprocessing import LabelEncoder

print("Before encoding: ")
print(y[100:110])

labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)

print("\nAfter encoding: ")
print(y[100:110])

Before encoding: 
100    M
101    B
102    B
103    B
104    B
105    M
106    B
107    B
108    M
109    B
Name: diagnosis, dtype: object

After encoding: 
[1 0 0 0 0 1 0 0 1 0]


<h1> Splitting Dataset </h1>

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

<h1> Features Scaling </h1>
Proses selanjutnya adalah menerapkan penskalaan fitur. Penskalaan memastikan bahwa hanya karena beberapa fitur berukuran besar, model tidak akan menggunakannya sebagai prediktor utama ([Baca lebih lanjut] (https://stackoverflow.com/questions/26225344/why-feature-scaling))

In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

<h1> 2. Preparing ANN </h1>

<h1> Importing Keras dan Initializing ANN </h1> 
<h1> Building The Layers</h1>
Selanjutnya membangun Layers. Kita dapat bermain-main dan mengubah jumlah unit tetapi jika kita tidak yakin dengan nomor apa yang akan diinisialisasi maka cukup menginisialisasi unit dari semua lapisan kecuali yang terakhir dengan (jumlah fitur + jumlah node output) / 2 yang sama dengan 15 dalam kasus kami. Hasil saya ditingkatkan dengan mengatur unit = 16 untuk lapisan pertama dan mengurangi unit di lapisan tersembunyi. Juga kita harus memberikan dimensi masukan untuk lapisan pertama saja. 'relu' mengacu pada unit linier yang diperbaiki dan sigmoid mengacu pada fungsi aktivasi sigmoid. Dengan bantuan fungsi aktivasi sigmoid, kita bisa mendapatkan kemungkinan klasifikasi yang mungkin bermanfaat dalam beberapa kasus untuk melakukan penelitian lebih lanjut.

<h1>Tuning Hyper Parameters</h1>
Pertama, mari kita temukan parameter hyper menggunakan model mana yang dapat memberikan prediksi yang lebih akurat. Di sini saya akan menyesuaikan batch_size, epochs, dan pengoptimal. Ini akan membutuhkan waktu untuk berlari, jadi duduk dan rileks.

In [38]:
## I already found out the tuned hyper parameters so commenting the code.

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 30))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [1, 5],'epochs': [100, 120],'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/100
409/409 [==============================] - 0s 1ms/step - loss: 0.4911 - accuracy: 0.8826
Epoch 2/100
409/409 [==============================] - 0s 1ms/step - loss: 0.1031 - accuracy: 0.9707
Epoch 3/100
409/409 [==============================] - 0s 974us/step - loss: 0.0801 - accuracy: 0.9756
Epoch 4/100
409/409 [==============================] - 0s 988us/step - loss: 0.0695 - accuracy: 0.9829
Epoch 5/100
409/409 [==============================] - 0s 971us/step - loss: 0.0627 - accuracy: 0.9853
Epoch 6/100
409/409 [==============================] - 0s 1ms/step - loss: 0.0578 - accuracy: 0.9878
Epoch 7/100
409/409 [==============================] - 0s 990us/step - loss: 0.0546 - accuracy: 0.9878
Epoch 8/100
409/409 [==============================] - 0s 1ms/step - loss: 0.0517 - accuracy: 0.9878
Epoch 9/100
409/409 [==============================] - 0s 1ms/step - loss: 0.0492 - accuracy: 0.9878
Epoch 10/100
409/409 [==============================] - 0s 983us/step - loss: 0.046

Setelah menjalankan sel di atas, saya mendapatkan hasil sebagai berikut :

In [39]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("best_parameters: ")
print(best_parameters)
print("\nbest_accuracy: ")
print(best_accuracy)

best_parameters: 
{'batch_size': 5, 'epochs': 120, 'optimizer': 'rmsprop'}

best_accuracy: 
0.9781159420289856


Sekarang mari membangun jaringan saraf dengan parameter yang kita temukan sebelumnya

In [20]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 30))
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

### Library Keras
Keras adalah high-level neural network API yang dikembangkan dengan Python dengan fokus tujuan untuk mempercepat proses riset atau percobaan

<h1> Compiling ANN </h1>

Mengompilasi pengklasifikasi. Menggunakan adam optimizer. Menggunakan binary_crossentropy untuk fungsi kerugian karena klasifikasinya adalah biner, yaitu hanya dua kelas 'M' atau 'B'.

In [21]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

<h1> Fitting the Data </h1>

Sekarang mari menyesuaikan datanya. Saya melatihnya dengan ukuran batch 1 dan 50 epoch dan akurasi pelatihan 99,12% dan prediksi akhir akurat 96,49%.

In [23]:
classifier.fit(X_train, y_train, batch_size = 1, epochs = 50)

Epoch 1/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0550 - accuracy: 0.9890
Epoch 2/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0412 - accuracy: 0.9890
Epoch 3/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0464 - accuracy: 0.9890
Epoch 4/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0462 - accuracy: 0.9890
Epoch 5/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0494 - accuracy: 0.9890
Epoch 6/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0461 - accuracy: 0.9890
Epoch 7/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0531 - accuracy: 0.9890
Epoch 8/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0435 - accuracy: 0.9912
Epoch 9/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0550 - accuracy: 0.9890
Epoch 10/50
455/455 [==============================] - 1s 1ms/step - loss: 0.0496 - accuracy: 0.9890

<h1> Loading/ Saving The Model </h1>

In [27]:
from keras.models import load_model

classifier.save('breast_cancer_model.h5') #Save trained ANN
classifier = load_model('breast_cancer_model.h5')  #Load trained ANN

<h1> 3. Making Prediction </h1>

Sekarang y_pred berisi kemungkinan tumor berjenis Ganas atau Jinak. Kami akan menetapkan hasil benar atau salah berdasarkan probabilitasnya (jika probabilitas> = 0,5 daripada benar atau salah)

In [28]:
y_pred = classifier.predict(X_test)
y_pred = [ 1 if y>=0.5 else 0 for y in y_pred ]

Sekarang mari kita cetak matriks untuk memeriksa akurasi kita.

In [29]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
print("Accuracy: "+ str(accuracy*100)+"%")

[[65  2]
 [ 1 46]]
Accuracy: 97.36842105263158%
